In [1]:
import csv
import io
import sys
import asyncio

# Pastikan file RAG.py ada di folder yang sama dan memiliki fungsi/chain yang sesuai
from RAG import rrf_retriever_chain, rrf_retriever_chain_gemma, rrf_retriever_chain_gemma3_12b

# ==========================================
# 2. DATA MENTAH (RAW DATA) - 130 Pertanyaan
# ==========================================
csv_content = """question,ground_truth
"Apa fungsi utama dari aplikasi SPK-Driver bagi pengguna?","Fungsi utama aplikasi SPK-Driver adalah untuk mempermudah pencatatan pembelian BBM dengan rekanan SPBU."
"Melalui platform apa pengguna dapat mengunduh aplikasi SPK-Driver?","Pengguna dapat mengunduh aplikasi SPK-Driver melalui Google Play Store pada tautan play.google.com yang disediakan."
"Mengapa aplikasi SPK-Driver memerlukan izin akses lokasi di latar belakang (background location)?","Aplikasi SPK-Driver memerlukan izin akses lokasi di latar belakang untuk pelacakan lokasi selama tugas SPK, pengukuran kinerja, pencegahan kecurangan, dan pelaporan keuangan universitas."
"Langkah apa yang harus dilakukan pengguna aplikasi SPK-Driver setelah memilih item SPK dari daftar?","Setelah memilih SPK dari daftar, pengguna harus mengklik tombol 'BUAT QR' untuk memproses transaksi."
"Pada kondisi apa pengguna aplikasi SPK-Driver diperbolehkan menekan tombol 'Selesaikan SPK'?","Pengguna boleh menekan tombol 'Selesaikan SPK' setelah transaksi selesai dilakukan dan posisi driver sudah kembali ke lingkungan Undiksha."
"Apa saja menu utama yang ditampilkan pada beranda aplikasi SPK-Driver?","Menu utama yang tampil di beranda aplikasi SPK-Driver meliputi Daftar SPK, Riwayat Perjalanan, Notifikasi, dan Profile."
"Informasi detail apa saja yang muncul pada layar aplikasi SPK-Driver setelah transaksi berhasil?","Setelah transaksi berhasil, informasi yang muncul pada layar detail meliputi Nopol, Jenis BBM, Jumlah Liter, dan Nominal Bayar."
"Unit mana yang mengembangkan aplikasi SPK-Driver?","Aplikasi SPK-Driver dikembangkan oleh unit Teknologi Informasi & Komunikasi UNDIKSHA."
"Apa indikator visual yang menandakan bahwa data transaksi berhasil ditambahkan dalam aplikasi SPK-Driver?","Indikator bahwa data transaksi berhasil ditambahkan adalah munculnya popup konfirmasi dengan tanda centang hijau bertuliskan 'data berhasil ditambahkan!'."
"Apakah data lokasi pengguna aplikasi SPK-Driver dibagikan kepada pihak ketiga?","Tidak, data lokasi pengguna aplikasi SPK-Driver disimpan di server Undiksha dan tidak dibagikan kepada siapapun."
"Berapa total alokasi belanja negara yang ditetapkan dalam APBN Tahun Anggaran 2025?","Total alokasi belanja negara yang ditetapkan dalam APBN Tahun Anggaran 2025 adalah sebesar Rp3.621,3 triliun."
"Berapa nilai defisit anggaran yang tercatat dalam dokumen APBN Tahun Anggaran 2025?","Defisit anggaran yang tercatat dalam APBN Tahun Anggaran 2025 adalah sebesar Rp616,2 triliun."
"Bagaimana arahan Presiden mengenai pengelolaan APBN 2025 di tengah kondisi geopolitik saat ini?","Presiden mengarahkan agar APBN TA 2025 dikelola dengan prinsip prudent, disiplin, dan tepat sasaran untuk menjaga stabilitas, inklusifitas, dan keberlanjutan di tengah ketidakpastian geopolitik."
"Apa saja jenis pengeluaran non-prioritas yang diminta untuk dikurangi dalam pelaksanaan anggaran 2025?","Pengeluaran non-prioritas yang harus dikurangi meliputi kegiatan yang bersifat seremoni, kajian, dan seminar."
"Apa instruksi Kementerian Keuangan kepada Kementerian/Lembaga terkait proses kontrak barang/jasa sebelum adanya kebijakan strategis baru?","Kementerian/Lembaga diinstruksikan untuk melakukan penundaan sementara terhadap proses perikatan atau kontrak barang/jasa, terutama untuk jenis belanja barang dan belanja modal."
"Tanggal berapa Presiden menyerahkan DIPA dan Buku Daftar Alokasi TKD TA 2025 secara digital?","Presiden menyerahkan DIPA dan Buku Daftar Alokasi TKD TA 2025 secara digital pada tanggal 10 Desember 2024."
"Siapa pejabat yang menandatangani surat S-27/PB/2025 secara elektronik?","Surat S-27/PB/2025 ditandatangani secara elektronik oleh Direktur Jenderal Perbendaharaan, Astera Primanto Bhakti."
"Apa tujuan utama pemerintah mendorong digitalisasi berdasarkan surat S-27/PB/2025?","Pemerintah mendorong digitalisasi dengan tujuan untuk meningkatkan efisiensi dan transparansi demi terciptanya pemerintahan yang bersih."
"Apa tindakan yang diminta dari Kementerian/Lembaga untuk mendukung program prioritas pemerintah dalam surat ini?","Kementerian/Lembaga diminta untuk meningkatkan sinergi dan harmonisasi kebijakan serta mengoptimalkan penggunaan anggaran."
"Kapan surat S-27/PB/2025 tentang Dukungan Pelaksanaan Kebijakan Pemerintah diterbitkan?","Surat S-27/PB/2025 diterbitkan pada tanggal 20 Januari 2025."
"Apa kriteria 'Jurnal Ilmiah Internasional Bereputasi' menurut SK Rektor Nomor 544/UN48/DT/2025?","Kriteria Jurnal Ilmiah Internasional Bereputasi adalah jurnal yang terindeks Scopus dan/atau Web of Science dengan impact factor lebih dari atau sama dengan 0,05 atau Scimago Journal Rank (SJR) lebih dari atau sama dengan 0,1."
"Berapa jumlah minimal sitasi terhadap publikasi dosen Undiksha yang wajib dilakukan oleh mahasiswa Program Sarjana?","Mahasiswa Program Sarjana wajib melakukan minimal 7 sitasi terhadap publikasi dosen Undiksha."
"Kapan ketentuan dalam SK Rektor Nomor 544/UN48/DT/2025 mulai berlaku bagi mahasiswa Program Magister dan Doktor?","Bagi mahasiswa Program Magister dan Doktor, SK Rektor Nomor 544/UN48/DT/2025 mulai berlaku pada yudisium setelah wisuda periode Maret 2025."
"Apa kewajiban publikasi artikel ilmiah bagi mahasiswa Program Doktor di Undiksha?","Mahasiswa Program Doktor wajib memiliki publikasi artikel ilmiah paling rendah pada jurnal internasional bereputasi terindeks Scopus (SJR >= 0,1) atau Web of Science (JIF >= 0,05)."
"Di mana tingkat minimal publikasi yang diwajibkan bagi mahasiswa Program Sarjana/Sarjana Terapan Undiksha?","Mahasiswa Program Sarjana/Sarjana Terapan wajib memiliki publikasi paling rendah pada jurnal ilmiah Sinta 6 atau prosiding konferensi internasional."
"Siapa yang harus disertakan dalam publikasi artikel ilmiah yang berasal dari hasil karya akhir (skripsi/tesis/disertasi)?","Publikasi yang berasal dari hasil karya akhir wajib menyertakan pembimbing utama dan pembimbing pendamping, atau sesuai kesepakatan."
"Apa kewajiban terkait Kekayaan Intelektual (KI) bagi mahasiswa Program Diploma 3?","Mahasiswa Program Diploma 3 diwajibkan memiliki minimal hak kekayaan intelektual berupa hak cipta."
"Berapa SKS terdaftar publikasi dosen yang wajib disitasi oleh mahasiswa Program Doktor dalam disertasinya?","Mahasiswa Program Doktor wajib melakukan sitasi minimal 15 SKS terdaftar publikasi dosen Universitas yang relevan."
"Bagaimana posisi urutan nama mahasiswa dalam publikasi yang digunakan sebagai syarat yudisium?","Dalam publikasi yang digunakan sebagai syarat yudisium, mahasiswa harus tercatat sebagai penulis pertama."
"Berapa nomor SK Rektor yang mengatur tentang Kewajiban Publikasi, Kekayaan Intelektual, dan Sitasi tahun 2025?","Aturan tentang Kewajiban Publikasi, Kekayaan Intelektual, dan Sitasi tahun 2025 tertuang dalam SK Rektor Nomor 544/UN48/DT/2025."
"Apa alamat URL yang digunakan mahasiswa Undiksha untuk login ke sistem SIMKATMAWA?","Mahasiswa dapat melakukan login ke SIMKATMAWA melalui alamat URL sso.undiksha.ac.id."
"Siapa pihak yang melakukan verifikasi tahap pertama setelah mahasiswa menginput data prestasi di SIMKATMAWA?","Pihak yang melakukan verifikasi tahap pertama setelah input data adalah Operator Fakultas."
"Apa saja empat submenu yang tersedia di dalam menu 'Prestasi' pada SIMKATMAWA?","Empat submenu yang terdapat dalam menu Prestasi adalah Lomba Mandiri, Rekognisi, Sertifikasi, dan Wirausaha."
"Berapa batas maksimal ukuran file bukti prestasi yang dapat diunggah ke SIMKATMAWA?","Batas maksimal ukuran file yang dapat diunggah sebagai bukti prestasi adalah 2Mb."
"Format file apa yang diwajibkan untuk unggahan sertifikat atau bukti kegiatan di SIMKATMAWA?","Format file yang diterima di SIMKATMAWA untuk unggahan sertifikat atau bukti kegiatan adalah *.pdf."
"Bagaimana format penulisan anggota kelompok saat menginput Prestasi Lomba Mandiri di SIMKATMAWA?","Anggota kelompok diinput dengan memasukkan NIM seluruh anggota yang dipisahkan dengan tanda koma, dimana NIM urutan pertama adalah ketua kelompok."
"Apa langkah selanjutnya dalam sistem setelah Operator Fakultas memverifikasi data prestasi dengan status 'Sesuai'?","Setelah Operator Fakultas menyatakan data 'Sesuai', Operator Rektorat akan melaporkan data tersebut ke pusat."
"Apa yang harus dilakukan mahasiswa jika status ajuan di SIMKATMAWA adalah 'Menunggu verifikasi' namun terdapat komentar perbaikan?","Jika ada komentar perbaikan pada status 'Menunggu verifikasi' di SIMKATMAWA mahasiswa harus mengklik tombol 'Edit Ajuan' untuk memperbaiki data."
"Data detail apa saja yang wajib diisi mahasiswa pada form 'Lomba Mandiri'?","Data yang wajib diisi pada form lomba mandiri meliputi Nama Lomba, Penyelenggara, Jenis Kepesertaan, Kategori, Jumlah PT, Jumlah Negara (jika Internasional), Jumlah Peserta, Capaian Prestasi, serta Tanggal Mulai dan Akhir."
"Siapa pejabat yang berwenang mencairkan dana proposal kegiatan setelah disetujui PPK?","Pejabat yang berwenang melakukan pencairan dana setelah persetujuan PPK adalah Bendahara."
"Siapa nama mahasiswa yang ditetapkan sebagai Ketua Duta Keterbukaan Informasi Publik (KIP) Mahasiswa Undiksha Tahun 2025?","Mahasiswa yang ditetapkan sebagai Ketua Duta Keterbukaan Informasi Publik (KIP) Mahasiswa Undiksha Tahun 2025 adalah Komang Danda Widya Anugrah."
"Berapa nomor Keputusan Rektor yang menetapkan Duta KIP Mahasiswa Undiksha Tahun 2025?","Penetapan Duta KIP Mahasiswa Tahun 2025 diatur dalam Keputusan Rektor Nomor 814/UN48/KM.05.04/2025."
"Siapa saja dosen yang ditunjuk sebagai Pendamping Duta KIP Mahasiswa Tahun 2025?","Dosen yang ditunjuk sebagai Pendamping Duta KIP Mahasiswa Tahun 2025 adalah Dr. Kadek Wirahyuni, S.Pd., M.Pd. dan I Kadek Edi Yudiana, M.Pd."
"Siapa nama perwakilan mahasiswa dari Fakultas Kedokteran (FK) yang menjadi anggota Duta KIP 2025?","Perwakilan mahasiswa dari Fakultas Kedokteran (FK) yang menjadi anggota Duta KIP 2025 adalah Nadita Sulastri Br Purba dan Gede Pradnyana Putra."
"Dari mana sumber pembiayaan untuk pelaksanaan kegiatan Duta KIP Mahasiswa Tahun 2025?","Biaya pelaksanaan kegiatan dibebankan pada Anggaran DIPA Universitas Pendidikan Ganesha Tahun 2025 Nomor SP DIPA-139.03.2.693393/2025."
"Siapa mahasiswa dari Prodi Pendidikan Teknik Informatika yang terdaftar sebagai anggota Duta KIP 2025?","Mahasiswa dari Prodi Pendidikan Teknik Informatika yang menjadi anggota adalah Gede Ngurah Dananjaya Kajeng."
"Pada tanggal berapa Surat Keputusan penetapan Duta KIP Mahasiswa 2025 ini ditetapkan?","Surat Keputusan penetapan Duta KIP Mahasiswa 2025 ditetapkan di Singaraja pada tanggal 17 Maret 2025."
"Siapa yang menjabat sebagai Pengarah dalam struktur organisasi Duta KIP Mahasiswa Tahun 2025?","Pejabat yang menjabat sebagai Pengarah adalah Prof. Dr. I Wayan Lasmawan, M.Pd."
"Kepada siapa Duta KIP Mahasiswa bertanggung jawab dalam menjalankan tugasnya?","Duta KIP Mahasiswa bertanggung jawab langsung kepada Rektor."
"Siapa saja mahasiswa perwakilan Fakultas Hukum dan Ilmu Sosial (FHIS) dalam Duta KIP 2025?","Perwakilan mahasiswa dari FHIS adalah Gede Eka Sidi Artama dan Kunti Afrida Maharani."
"Siapa saja pejabat negara yang menjadi tujuan instruksi dalam Instruksi Presiden Nomor 1 Tahun 2025?","Instruksi Presiden Nomor 1 Tahun 2025 ditujukan kepada Para Menteri Kabinet Merah Putih, Panglima TNI, Kapolri, Jaksa Agung, Kepala LPNK, Pimpinan Kesekretariatan Lembaga Negara, Para Gubernur, dan Para Bupati/Wali Kota."
"Berapa persentase efisiensi belanja perjalanan dinas yang diinstruksikan Presiden kepada Gubernur dan Bupati/Wali Kota?","Presiden menginstruksikan Gubernur dan Bupati/Wali Kota untuk melakukan efisiensi belanja perjalanan dinas sebesar minimal 50% (lima puluh persen)."
"Berapa nilai penyesuaian untuk Dana Desa yang ditetapkan dalam rangka efisiensi Transfer ke Daerah?","Penyesuaian untuk Dana Desa dalam rangka efisiensi ditetapkan sebesar Rp2.000.000.000.000,00 (dua triliun rupiah)."
"Apa tindakan teknis yang harus dilakukan Menteri Keuangan terhadap anggaran K/L dalam DIPA sesuai Inpres ini?","Menteri Keuangan diinstruksikan untuk melakukan revisi anggaran K/L dalam DIPA dengan cara memblokir anggaran dan mencantumkannya pada catatan halaman IV A DIPA."
"Siapa pejabat yang bertugas memantau pelaksanaan efisiensi belanja oleh Pemerintah Daerah?","Pejabat yang bertugas memantau efisiensi belanja oleh Gubernur dan Bupati/Wali Kota adalah Menteri Dalam Negeri."
"Berapa total nilai efisiensi yang berasal dari Dana Otonomi Khusus?","Total nilai efisiensi yang berasal dari Dana Otonomi Khusus adalah sebesar Rp509.455.378.000,00."
"Mulai kapan Instruksi Presiden Nomor 1 Tahun 2025 ini dinyatakan berlaku?","Instruksi Presiden Nomor 1 Tahun 2025 mulai berlaku pada tanggal dikeluarkan, yaitu 22 Januari 2025."
"Siapa yang menandatangani naskah Instruksi Presiden Nomor 1 Tahun 2025?","Instruksi Presiden Nomor 1 Tahun 2025 ditandatangani oleh Presiden Republik Indonesia, Prabowo Subianto."
"Jenis kegiatan apa saja yang belanjanya harus dibatasi oleh Pemerintah Daerah sesuai Diktum Keempat Inpres ini?","sesuai Diktum Keempat Inpres, Pemerintah Daerah harus membatasi belanja untuk kegiatan yang bersifat seremonial, kajian, studi banding, pencetakan, publikasi, dan seminar/focus group discussion."
"Lembaga mana yang ditugaskan untuk mengawasi pelaksanaan Instruksi Presiden Nomor 1 Tahun 2025?","Badan Pengawasan Keuangan dan Pembangunan (BPKP) adalah lembaga yang ditugaskan untuk mengawasi pelaksanaan Instruksi Presiden Nomor 1 Tahun 2025."
"Berapa total target efisiensi anggaran belanja seluruh Kementerian/Lembaga untuk Tahun Anggaran 2025?","Total target efisiensi anggaran belanja seluruh Kementerian/Lembaga Tahun Anggaran 2025 adalah sebesar Rp256.100.000.000.000,00 (dua ratus lima puluh enam triliun seratus miliar rupiah)."
"Apa dua jenis belanja utama yang dikecualikan dari identifikasi rencana efisiensi K/L?","Dua jenis belanja yang dikecualikan dari rencana efisiensi K/L adalah Belanja Pegawai dan Belanja Bantuan Sosial."
"Berapa persentase target penghematan untuk akun belanja 'Perjalanan Dinas' menurut surat efisiensi KL?","Target penghematan untuk akun belanja Perjalanan Dinas menurut surat efisiensi KL ditetapkan sebesar 53,9%."
"Berapa persentase target efisiensi yang ditetapkan untuk belanja 'Alat Tulis Kantor'?","Target efisiensi untuk belanja Alat Tulis Kantor ditetapkan sebesar 90,0%."
"Kapan batas waktu terakhir bagi Kementerian/Lembaga untuk menyampaikan usulan revisi anggaran ke Menteri Keuangan?","Kementerian/Lembaga wajib menyampaikan usulan revisi anggaran paling lambat tanggal 14 Februari 2025."
"Apa konsekuensi jika Kementerian/Lembaga tidak menyampaikan usulan revisi anggaran hingga batas waktu 14 Februari 2025?","Jika Kementerian/Lembaga tidak menyampaikan usulan tepat waktu, Kementerian Keuangan c.q. Direktorat Jenderal Anggaran akan secara mandiri mencantumkan efisiensi dalam catatan halaman IV A DIPA."
"Kepada pihak mana Kementerian/Lembaga harus menyampaikan hasil identifikasi efisiensi untuk mendapat persetujuan sebelum diajukan ke Kemenkeu?","Hasil identifikasi efisiensi yang dilakukan Kementerian/Lembaga harus disampaikan terlebih dahulu kepada mitra Komisi Dewan Perwakilan Rakyat (DPR) untuk mendapatkan persetujuan."
"Bagaimana perlakuan efisiensi terhadap anggaran yang bersumber dari pinjaman dan hibah?","Efisiensi anggaran diprioritaskan pada sumber dana selain pinjaman dan hibah, kecuali jika kegiatan tersebut tidak dapat terlaksana hingga akhir tahun 2025."
"Berapa persentase penghematan yang ditargetkan untuk belanja Sewa Gedung, Kendaraan, dan Peralatan?","Persentase penghematan yang ditargetkan untuk belanja Sewa Gedung, Kendaraan, dan Peralatan adalah sebesar 73,3%."
"Siapa pejabat yang menandatangani surat instruksi efisiensi belanja S-37/MK.02/2025?","Surat instruksi efisiensi belanja S-37/MK.02/2025 ditandatangani oleh Menteri Keuangan, Sri Mulyani Indrawati."
"Siapa yang ditetapkan sebagai Ketua Duta Keterbukaan Informasi Publik (KIP) Alumni Undiksha periode 2025-2026?","Putu Paundra Ananda ditetapkan sebagai Ketua Duta KIP Alumni Undiksha untuk periode 2025-2026."
"Berapa lama durasi masa bhakti Duta KIP Alumni Undiksha sesuai SK Nomor 815/UN48/KM/2025?","Masa bhakti Duta KIP Alumni Undiksha sesuai SK Nomor 815/UN48/KM/2025 adalah selama dua tahun, yaitu tahun 2025 sampai dengan 2026."
"Siapa saja alumni dari Fakultas Ekonomi (FE) yang menjadi anggota Duta KIP Alumni 2025-2026?","Alumni dari Fakultas Ekonomi yang menjadi anggota antara lain Amanda Beta Ardelia Putri, Kadek Yuda Arimbawa, dan Ketut Mudi Ari Anggara."
"Siapa nama alumni dari Prodi Pendidikan Bahasa Inggris yang terdaftar sebagai anggota Duta KIP Alumni?","Alumni dari Prodi Pendidikan Bahasa Inggris yang menjadi anggota adalah Luh Nityaswari Laba dan Ni Luh Putu Diah Raspadewi."
"Siapa Dosen Pendamping yang ditunjuk untuk mendampingi Duta KIP Alumni Undiksha?","Dosen Pendamping untuk mendampingi Duta KIP Alumni Undiksha yang ditunjuk adalah Dr. Kadek Wirahyuni, S.Pd., M.Pd. dan I Kadek Edi Yudiana, M.Pd."
"Siapa yang menjabat sebagai Penanggung jawab pertama dalam struktur Duta KIP Alumni Undiksha?","Prof. Dr. Gede Rasben Dantes, S.T., M.T.I. menjabat sebagai Penanggung jawab pertama dalam struktur tersebut."
"Tanggal berapa SK penetapan Duta KIP Alumni Undiksha ini ditandatangani?","SK penetapan Duta KIP Alumni Undiksha ditandatangani pada tanggal 17 Maret 2025."
"Siapa nama alumni dari Prodi Pendidikan Fisika yang menjadi anggota Duta KIP Alumni?","Alumni dari Prodi Pendidikan Fisika yang menjadi anggota adalah Anak Agung Gde Suyoga Wiguna dan I Komang Restu Widi Artha."
"Dari fakultas mana Kadek Dedy Teriana, salah satu anggota Duta KIP Alumni, berasal?","Kadek Dedy Teriana berasal dari Fakultas Olahraga dan Kesehatan (FOK)."
"Apa peran Prof. Dr. I Wayan Lasmawan, M.Pd. dalam struktur organisasi Duta KIP Alumni?","Prof. Dr. I Wayan Lasmawan, M.Pd. memiliki peran sebagai Pengarah dalam struktur Duta KIP Alumni."
"Berapa total nilai efisiensi belanja nasional (Gabungan K/L dan TKD) yang tercantum dalam surat dukungan kebijakan Inpres No. 1 Tahun 2025?","Total nilai efisiensi belanja nasional sesuai surat dukungan kebijakan Inpres No. 1 Tahun 2025 adalah sebesar Rp306.695.177.420.000,-."
"Berapa nilai efisiensi spesifik untuk pos Transfer ke Daerah (TKD) pada Tahun Anggaran 2025?","Nilai efisiensi untuk pos Transfer ke Daerah (TKD) adalah sebesar Rp50.595.177.420.000,-."
"Bagaimana instruksi terkait kontrak/perikatan yang sudah terlanjur dibuat sebelum keluarnya Inpres No. 1 Tahun 2025?","Kontrak atau perikatan yang sudah dibuat sebelum Inpres keluar dapat tetap dilaksanakan sesuai ketentuan, namun harus tetap memperhatikan kebijakan efisiensi belanja."
"Apa syarat bagi rencana perikatan/kontrak belanja barang dan modal yang baru akan dilakukan oleh K/L?","Rencana perikatan/kontrak baru harus memastikan adanya perhatian terhadap kebijakan efisiensi belanja dan pemenuhan target efisiensi."
"Surat S-37/PB/2025 ini merupakan tindak lanjut atau susulan dari surat nomor berapa?","Surat S-37/PB/2025 ini menyusuli surat sebelumnya yaitu nomor S-27/PB/2025 tentang Dukungan Pelaksanaan Kebijakan Pemerintah."
"Kapan surat S-37/PB/2025 tentang Dukungan Pelaksanaan Kebijakan Efisiensi Belanja diterbitkan?","Surat S-37/PB/2025 tentang Dukungan Pelaksanaan Kebijakan Efisiensi Belanja diterbitkan pada tanggal 31 Januari 2025."
"Apa tujuan utama dari koordinasi perikatan/kontrak yang dilakukan oleh satuan kerja?","Koordinasi perikatan/kontrak bertujuan untuk memenuhi upaya efisiensi belanja sebagaimana diamanatkan dalam Instruksi Presiden Nomor 1 Tahun 2025."
"Siapa pihak penerima yang dituju dalam surat S-37/PB/2025 ini?","Surat S-37/PB/2025 ditujukan kepada Sekretaris Jenderal/Sekretaris Utama/Sekretaris Kementerian/Lembaga."
"Berapa angka target efisiensi belanja Kementerian/Lembaga (K/L) yang ditegaskan kembali dalam surat S-37/PB/2025?","Surat S-37/PB/2025 menegaskan kembali bahwa target efisiensi belanja K/L adalah sebesar Rp256.100.000.000.000,-."
"Siapa pejabat yang menandatangani surat S-37/PB/2025?","Surat S-37/PB/2025 ditandatangani oleh Direktur Jenderal Perbendaharaan, Astera Primanto Bhakti."
"Apa saja tiga kategori mahasiswa yang berhak melamar Beasiswa Ganesha Tahun 2025?","Tiga kategori yang berhak melamar Beasiswa Ganesha Tahun 2025 adalah mahasiswa tidak mampu secara ekonomi, mahasiswa berprestasi, dan mahasiswa yang aktif berkegiatan di Organisasi Mahasiswa (BEM REMA)."
"Berapa standar minimal IPK yang harus dimiliki pelamar Beasiswa Ganesha kategori Mahasiswa Tidak Mampu?","Pelamar kategori Mahasiswa Tidak Mampu harus memiliki IPK minimal 3,00 pada semester ganjil Tahun Akademik 2024/2025."
"Berapa batas minimal IPK bagi pelamar Beasiswa Ganesha kategori Mahasiswa Berprestasi?","Pelamar kategori Mahasiswa Berprestasi harus memiliki IPK minimal 3,50 pada semester ganjil Tahun Akademik 2024/2025."
"Sampai semester berapa batas tertinggi mahasiswa S1 diperbolehkan melamar Beasiswa Ganesha?","Mahasiswa S1 diperbolehkan melamar Beasiswa Ganesha paling tinggi saat duduk pada semester VI."
"Dokumen apa saja yang diperlukan untuk membuktikan kondisi ekonomi bagi pelamar kategori tidak mampu?","Bukti kondisi ekonomi harus berupa Surat Keterangan Tidak Mampu (SKTM) dan surat keterangan penghasilan orang tua yang dikeluarkan oleh kelurahan atau desa setempat."
"Dokumen khusus apa yang wajib dilampirkan oleh pelamar beasiswa kategori aktif organisasi (BEM REMA)?","Pelamar kategori aktif organisasi wajib melampirkan Surat Keputusan Rektor yang menyatakan keaktifan mereka dalam kegiatan MPM, BEM REMA, atau BEM Fakultas masa bhakti 2024/2025."
"Berapa rentang periode validitas sertifikat prestasi yang dapat digunakan untuk melamar Beasiswa Ganesha 2025?","Sertifikat prestasi yang berlaku untuk pendaftaran adalah yang diperoleh mulai Bulan Mei 2024 sampai dengan Mei 2025."
"Kepada siapa surat permohonan Beasiswa Ganesha harus ditujukan?","Surat permohonan Beasiswa Ganesha harus ditujukan kepada Rektor, u.p (untuk perhatian) Wakil Rektor III."
"Aspek kepribadian apa saja yang dinilai dalam lembar penilaian pelamar Beasiswa Ganesha?","Aspek kepribadian yang dinilai meliputi Kejujuran, Kepemimpinan, Tanggung Jawab, Etika/Sopan Santun, Hubungan Sosial, dan Kerajinan."
"Siapa pihak yang berwenang menandatangani Lembar Penilaian Kepribadian Calon Pelamar beasiswa?","Lembar Penilaian Kepribadian Calon Pelamar beasiswa harus ditandatangani oleh Koordinator Program Studi."
"Siapa mahasiswa yang ditetapkan sebagai Ketua Duta Keterbukaan Informasi Publik (KIP) Undiksha Tahun 2024?","Ketua Duta Keterbukaan Informasi Publik (KIP) Undiksha Tahun 2024 yang ditetapkan adalah Cahya Palasari."
"Berapa nomor Keputusan Rektor yang mengatur tentang Duta KIP Undiksha Tahun 2024?","Keputusan Rektor tentang Duta KIP Undiksha Tahun 2024 adalah Nomor 2525/UN48/KM/2024."
"Siapa nama mahasiswa anggota Duta KIP Undiksha 2024 yang berasal dari Prodi Kedokteran?","Mahasiswa anggota Duta KIP Undiksha 2024 yang berasal dari Prodi Kedokteran adalah I Putu Duta Mahardika dan Ketut Lingga Amritiya."
"Kapan Keputusan Rektor Nomor 2525/UN48/KM/2024 tentang Duta KIP ditetapkan?","Keputusan Rektor Nomor 2525/UN48/KM/2024 ditetapkan di Singaraja pada tanggal 3 September 2024."
"Siapa mahasiswa dari Prodi Akuntansi (FE) yang menjadi anggota Duta KIP 2024?","Mahasiswa dari Prodi Akuntansi (FE) yang menjadi anggota Duta KIP 2024 adalah Kadek Ayu Puja Dewanti dan Giokvannie Vernaya Tanty."
"Siapa saja Dosen Pendamping Duta KIP Undiksha Tahun 2024 sesuai SK Nomor 2525/UN48/KM/2024?","Dosen Pendamping Duta KIP Undiksha Tahun 2024 sesuai SK tersebut adalah Dr. Kadek Wirahyuni, S.Pd., M.Pd. dan I Kadek Edi Yudiana, M.Pd."
"Siapa mahasiswa perwakilan dari Fakultas Hukum dan Ilmu Sosial (FHIS) dalam Duta KIP 2024?","Perwakilan mahasiswa dari Fakultas Hukum dan Ilmu Sosial (FHIS) dalam Duta KIP 2024 adalah I Made Gandy Gading Sanjaya dan Ni Nyoman Rita Tresnayani."
"Kepada siapa Duta KIP Undiksha Tahun 2024 bertanggung jawab?","Duta KIP Undiksha Tahun 2024 bertanggung jawab kepada Rektor."
"Siapa nama anggota Duta KIP 2024 yang berasal dari Prodi Pendidikan Bahasa Inggris?","Anggota Duta KIP 2024 yang berasal dari Prodi Pendidikan Bahasa Inggris adalah Komang Devi Damayanti."
"Dari anggaran mana biaya pelaksanaan kegiatan Duta KIP 2024 dibebankan?","Biaya pelaksanaan kegiatan Duta KIP 2024 dibebankan pada Anggaran DIPA Universitas Pendidikan Ganesha Tahun 2024."
"Berapa nomor SK Rektor yang menetapkan Daftar Informasi Publik Undiksha Tahun 2024?","SK Rektor tentang Penetapan Daftar Informasi Publik Undiksha Tahun 2024 adalah Nomor 571/UN48/PT/2024."
"Berapa lama jangka waktu penyimpanan arsip untuk Laporan Keuangan Undiksha Audited?","Jangka waktu penyimpanan arsip untuk Laporan Keuangan Undiksha Audited adalah Permanen."
"Apa jenis informasi yang wajib tersedia setiap saat terkait pimpinan universitas?","Informasi yang wajib tersedia setiap saat terkait pimpinan universitas meliputi agenda kerja pimpinan dan dokumen surat menyurat pimpinan dalam rangka pelaksanaan tugas pokok dan fungsi."
"Unit mana yang menjadi penanggung jawab pembuatan informasi mengenai Seleksi Penerimaan Mahasiswa Baru?","Penanggung jawab pembuatan informasi mengenai Seleksi Penerimaan Mahasiswa Baru adalah Biro Akademik dan Kerjasama (BAK)."
"Berapa lama jangka waktu penyimpanan (retensi) arsip untuk Informasi Lelang?","Jangka waktu penyimpanan (retensi) arsip untuk Informasi Lelang adalah 5 (lima) tahun, kemudian musnah."
"Unit mana yang menguasai informasi tentang Daftar Urut Kepangkatan (DUK) dosen dan pegawai?","Unit yang menguasai informasi tentang Daftar Urut Kepangkatan (DUK) dosen dan pegawai adalah Bagian Kepegawaian."
"Berapa lama durasi penyediaan informasi profil Universitas Pendidikan Ganesha secara berkala?","Informasi tentang profil Universitas Pendidikan Ganesha disediakan secara berkala selama 1 (satu) tahun."
"Bagaimana status daftar informasi yang dikecualikan di Undiksha pada tahun 2024 saat SK ditetapkan?","Status daftar informasi yang dikecualikan di Undiksha pada tahun 2024 adalah sampai saat SK ditetapkan, belum ada informasi dikecualikan yang sudah habis jangka waktu pengecualiannya sebagai informasi terbuka."
"Berapa lama informasi tentang Peta Kampus disimpan sebelum dinilai kembali?","Informasi tentang Peta Kampus disimpan selama 10 (sepuluh) tahun kemudian dinilai kembali."
"Siapa pihak yang bertanggung jawab menerbitkan informasi tentang Rencana Strategis (Renstra)?","Pihak yang bertanggung jawab menerbitkan informasi tentang Rencana Strategis (Renstra) adalah Biro Perencanaan dan Kemahasiswaan."
"Siapa Ketua Tim Pejabat Pengelola Informasi dan Dokumentasi (PPID) Undiksha Tahun 2023?","Ketua Tim PPID Undiksha Tahun 2023 adalah Prof. Dr. Drs. I Ketut Sudiana, M.Kes."
"Berapa nomor SK Rektor yang menetapkan Tim PPID Undiksha Tahun 2023?","SK Rektor tentang Tim PPID Undiksha Tahun 2023 adalah Nomor 3299/UN48/PT/2023."
"SK nomor berapa yang dicabut dan dinyatakan tidak berlaku dengan terbitnya SK Nomor 3299/UN48/PT/2023?","Dengan terbitnya SK Nomor 3299/UN48/PT/2023, maka Keputusan Rektor Nomor 2413/UN48/PT/2023 tanggal 31 Juli 2023 dicabut dan dinyatakan tidak berlaku."
"Siapa saja personel yang bertanggung jawab di bidang TIK dalam Tim PPID 2023?","Personel yang bertanggung jawab di bidang TIK dalam Tim PPID 2023 adalah I Ketut Resika Arthana, S.T., M.Kom., Ida Komang Widhiarjaya S.T., M.Pd., dan Made Yuda Sadewa S.Pd."
"Siapa saja anggota tim Penanganan Pengaduan Rektorat dalam struktur PPID 2023?","Anggota tim Penanganan Pengaduan Rektorat meliputi I Ketut Agus Supradnyan S.H., Putu Pebri Sumayanti S.E., I Gusti Agung Ngurah Pradipta Dwi Setiawan S.T., dan Kadek Ayu Partiwi S.Pd."
"Siapa yang menangani pengaduan untuk Fakultas Teknik dan Kejuruan (FTK) dalam Tim PPID 2023?","Penanganan pengaduan dalam Tim PPID 2023 untuk Fakultas Teknik dan Kejuruan (FTK) dilakukan oleh I Ketut Agus Angga Dharma, S.E. dan I Gst. Bgs. Ngr. Eman Sugiri, S.E."
"Siapa personel yang bertugas dalam bagian Penyelesaian Sengketa Tim PPID 2023?","Personel yang bertugas dalam Penyelesaian Sengketa Tim PPID 2023 adalah I Wayan Krisna Eka Putra, I Gede Putu Banu Astawa, Komang Endrawan Sumadi Putra, dan Si Ngurah Ardhya."
"Kapan SK Rektor Nomor 3299/UN48/PT/2023 tentang Tim PPID ini ditetapkan?","SK Rektor Nomor 3299/UN48/PT/2023 ditetapkan pada tanggal 15 Nopember 2023."
"Siapa yang menjabat sebagai Sekretaris I dalam struktur Tim PPID Undiksha 2023?","Sekretaris I dalam struktur Tim PPID Undiksha 2023 adalah Dr. Made Agus Wijaya, S.Pd., M.Pd."
"Dari sumber mana anggaran untuk kegiatan Tim PPID 2023 dibebankan?","Anggaran untuk kegiatan Tim PPID 2023 dibebankan pada Anggaran DIPA Universitas Pendidikan Ganesha Tahun 2023 Nomor SP DPA-023.17.2.677530/2023 revisi 10."
"""

# ==========================================
# 3. PROSES GENERASI DATASET
# ==========================================
async def generate_dataset():
    # Membaca string CSV mentah
    f_input = io.StringIO(csv_content.strip())
    reader = csv.DictReader(f_input)

    # List untuk menampung data hasil
    processed_rows = []

    print("Sedang memproses pertanyaan ke RAG...")

    # Iterate reader, tapi kita convert ke list dulu agar bisa enumerate
    data_list = list(reader)

    for i, row in enumerate(data_list, 1):
        q = row['question']
        print(f"\n[Pertanyaan {i}]: {q}")
        
        try:
            # --- 1. Memanggil fungsi RAG ---
            # Menggunakan await karena rrf_retriever_chain kemungkinan async
            result = await rrf_retriever_chain_gemma3_12b(q) 

            # --- 2. Ambil Output ---
            # Adaptasi pengambilan key dictionary sesuai output RAG Anda
            # Jika output chain adalah object 'Run' langchain, gunakan .get('output') dsb
            ans = result.get('answer', "No answer found")
            source_docs = result.get('source_documents', [])
            
            # --- 3. Ekstrak Konteks ---
            doc_contents = []
            print(f"Jawaban LLM: {ans}")
            print("-" * 40)
            
            for idx, doc in enumerate(source_docs, 1):
                text = ""
                score = "N/A"
                
                # Logika Deteksi Tipe Objek
                if hasattr(doc, 'payload'): # Jika dari Qdrant Native
                    text = doc.payload.get('text', '')
                    score = getattr(doc, 'score', 'N/A')
                elif hasattr(doc, 'page_content'): # Jika dari LangChain Document
                    text = doc.page_content
                    if hasattr(doc, 'metadata'):
                        score = doc.metadata.get('score', 'N/A')
                elif isinstance(doc, dict): # Jika dictionary
                    text = doc.get('text', str(doc))
                else:
                    text = str(doc)
                
                # Print Log (Optional)
                # print(f"   📄 [Dokumen {idx}] Score: {score}")
                
                doc_contents.append(text)
            
            if not doc_contents:
                doc_contents = ["No context retrieved"]
                print("Tidak ada dokumen yang ditemukan.")

            # --- 4. Masukkan ke Row ---
            row['answer'] = ans
            row['contexts'] = doc_contents # RAGAs butuh list of strings
            
            processed_rows.append(row)

        except Exception as e:
            print(f"Error memproses pertanyaan: {e}")
            row['answer'] = "Error during generation"
            row['contexts'] = []
            processed_rows.append(row)

    # ==========================================
    # 4. MENULIS KE FILE CSV FINAL
    # ==========================================
    filename = "dataset_ragas_lengkap.csv"
    fieldnames = ['question', 'answer', 'contexts', 'ground_truth']

    with open(filename, "w", encoding="utf-8", newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(processed_rows)

    print(f"\nSelesai! File '{filename}' berhasil dibuat.")

await generate_dataset()

Koneksi ke Firebase Firestore berhasil.
Inisialisasi LLM dengan model: meta-llama/llama-4-scout-17b-16e-instruct
Inisialisasi Evaluator dengan model: gpt-4o-mini
Running on: cuda
Loading Dense Model (Qwen 4-bit)...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Sparse Model (Splade v3)...
Sedang memproses pertanyaan ke RAG...

[Pertanyaan 1]: Apa fungsi utama dari aplikasi SPK-Driver bagi pengguna?
Jawaban LLM: Aplikasi SPK-Driver digunakan oleh komunitas Universitas Pendidikan Ganesha bersama mitra stasiun pengisian bahan bakar.
----------------------------------------

[Pertanyaan 2]: Melalui platform apa pengguna dapat mengunduh aplikasi SPK-Driver?
Jawaban LLM: Pengguna dapat mengunduh aplikasi SPK-Driver melalui platform Google Play Store.
----------------------------------------

[Pertanyaan 3]: Mengapa aplikasi SPK-Driver memerlukan izin akses lokasi di latar belakang (background location)?
Jawaban LLM: Aplikasi SPK-Driver merekam lokasi pengguna di latar belakang untuk laporan kinerja, pencegahan kecurangan, dan pelaporan keuangan universitas.
----------------------------------------

[Pertanyaan 4]: Langkah apa yang harus dilakukan pengguna aplikasi SPK-Driver setelah memilih item SPK dari daftar?
Jawaban LLM: Pengguna aplika

In [2]:
import pandas as pd
import os

# ==========================================
# KONFIGURASI FILE
# ==========================================
csv_filename = "dataset_ragas_lengkap.csv"   # File input
excel_filename = "dataset_ragas_lengkap.xlsx" # File output

# ==========================================
# PROSES KONVERSI & FORMATTING
# ==========================================
if os.path.exists(csv_filename):
    print(f"Membaca file: {csv_filename}...")
    
    # 1. Baca CSV ke DataFrame
    try:
        df_final = pd.read_csv(csv_filename)
        print("Berhasil membaca CSV.")
    except Exception as e:
        print(f"Gagal membaca CSV: {e}")
        exit()

    # 2. Setup Writer Excel
    print(f"Sedang membuat file Excel: {excel_filename}...")
    
    with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
        # Tulis DataFrame ke Excel (tanpa index)
        df_final.to_excel(writer, sheet_name='Sheet1', index=False)
        
        # Akses Workbook dan Worksheet untuk formatting
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        
        # --- DEFINISI FORMAT ---
        
        # Format isi tabel: Wrap text, Rata atas, Border tipis
        cell_format = workbook.add_format({
            'text_wrap': True,
            'valign': 'top',
            'border': 1
        })
        
        # Format Header: Bold, Background Hijau Muda, Border
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'fg_color': '#D7E4BC',
            'border': 1,
            'align': 'center'
        })

        # --- TERAPKAN LEBAR KOLOM ---
        # A: Question, B: Answer, C: Contexts, D: Ground Truth
        # Sesuaikan index kolom jika urutan di CSV berbeda
        
        # Kita loop kolom berdasarkan nama untuk memastikan urutan yang benar
        for col_num, col_name in enumerate(df_final.columns):
            # Tulis ulang header dengan format cantik
            worksheet.write(0, col_num, col_name, header_format)
            
            # Tentukan lebar kolom berdasarkan nama kolom
            if col_name == 'question':
                worksheet.set_column(col_num, col_num, 30, cell_format)
            elif col_name == 'answer':
                worksheet.set_column(col_num, col_num, 40, cell_format)
            elif col_name == 'contexts':
                worksheet.set_column(col_num, col_num, 60, cell_format) # Paling lebar
            elif col_name == 'ground_truth':
                worksheet.set_column(col_num, col_num, 40, cell_format)
            else:
                # Kolom lain (jika ada) default lebar 20
                worksheet.set_column(col_num, col_num, 20, cell_format)

    print(f"Selesai! File '{excel_filename}' berhasil dibuat dengan format rapi.")

else:
    print(f"File '{csv_filename}' tidak ditemukan di folder ini.")
    print("Pastikan nama filenya benar dan berada satu folder dengan script ini.")

Membaca file: dataset_ragas_lengkap.csv...
Berhasil membaca CSV.
Sedang membuat file Excel: dataset_ragas_lengkap.xlsx...
Selesai! File 'dataset_ragas_lengkap.xlsx' berhasil dibuat dengan format rapi.


In [4]:
import math

# --- Definisi Metode Integrasi Numerik ---

def trapezoidal_rule(f, a, b, n):
    h = (b - a) / n
    result = 0.5 * (f(a) + f(b))
    for i in range(1, n):
        result += f(a + i * h)
    return result * h

def simpsons_rule(f, a, b, n):
    if n % 2 != 0:
        return "N harus genap untuk aturan Simpson"
    h = (b - a) / n
    result = f(a) + f(b)
    for i in range(1, n):
        weight = 4 if i % 2 != 0 else 2
        result += weight * f(a + i * h)
    return (h / 3) * result

def midpoint_rule(f, a, b, n):
    h = (b - a) / n
    result = 0
    for i in range(n):
        result += f(a + (i + 0.5) * h)
    return result * h

# --- Eksekusi Validasi Soal ---

print("=== HASIL VALIDASI INTEGRASI NUMERIK ===")

# Soal 1
f1 = lambda x: 1/x
print(f"1) T5 (1 ke 2): {trapezoidal_rule(f1, 1, 2, 5):.4f}")

# Soal 2 & 3
f23 = lambda x: math.sqrt(4 + x**3)
print(f"2) T6 (0 ke 3): {trapezoidal_rule(f23, 0, 3, 6):.4f}")
print(f"3) S6 (0 ke 3): {simpsons_rule(f23, 0, 3, 6):.4f}")

# Soal 4
f4 = lambda x: x**2
print(f"4) M6 (0 ke 12): {midpoint_rule(f4, 0, 12, 6):.4f}")

# Soal 5
f5 = lambda x: (math.sin(math.pi * x))**2
print(f"5) M3 (0 ke 1): {midpoint_rule(f5, 0, 1, 3):.4f}")

# Soal 10 & 11
f1011 = lambda x: 1 / (1 + x**3)
print(f"10) T6 (0 ke 3): {trapezoidal_rule(f1011, 0, 3, 6):.4f}")
print(f"11) S6 (0 ke 3): {simpsons_rule(f1011, 0, 3, 6):.4f}")

# Soal 12 & 13
f1213 = lambda x: math.exp(-x**2)
print(f"12) T4 (0 ke 0.8): {trapezoidal_rule(f1213, 0, 0.8, 4):.4f}")
print(f"13) S4 (0 ke 0.8): {simpsons_rule(f1213, 0, 0.8, 4):.4f}")

# Soal 18 (Estimasi Pi)
f18 = lambda x: 1 / (1 + x**2)
val18 = trapezoidal_rule(f18, 0, 1, 4)
print(f"18) T4 (0 ke 1) * 4: {val18 * 4:.4f} (Estimasi Pi)")

# Soal 35 (Probability Density)
f35 = lambda x: (1 / math.sqrt(2 * math.pi)) * math.exp(-x**2 / 2)
print(f"35) S4 (0 ke 0.4): {simpsons_rule(f35, 0, 0.4, 4):.4f}")

# Soal 45 (Data Tabel)
x_coords = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
y_coords = [125, 125, 120, 112, 90, 90, 95, 88, 75, 35, 0]
h_table = 100
area_table = (h_table / 2) * (y_coords[0] + y_coords[-1] + 2 * sum(y_coords[1:-1]))
print(f"45) Luas Tanah (Tabel): {area_table:.2f} m^2")

=== HASIL VALIDASI INTEGRASI NUMERIK ===
1) T5 (1 ke 2): 0.6956
2) T6 (0 ke 3): 9.3304
3) S6 (0 ke 3): 9.2792
4) M6 (0 ke 12): 572.0000
5) M3 (0 ke 1): 0.5000
10) T6 (0 ke 3): 1.1533
11) S6 (0 ke 3): 1.1614
12) T4 (0 ke 0.8): 0.6549
13) S4 (0 ke 0.8): 0.6577
18) T4 (0 ke 1) * 4: 3.1312 (Estimasi Pi)
35) S4 (0 ke 0.4): 0.1554
45) Luas Tanah (Tabel): 89250.00 m^2


In [5]:
import numpy as np

def midpoint_rule(f, a, b, n):
    dx = (b - a) / n
    total = 0
    for i in range(n):
        mid = a + (i + 0.5) * dx
        total += f(mid)
    return total * dx

def trapezoidal_rule(f, a, b, n):
    dx = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (dx / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])

def simpsons_rule(f, a, b, n):
    if n % 2 != 0:
        return "N harus genap untuk aturan Simpson"
    dx = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return (dx / 3) * (y[0] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]) + y[-1])

# --- Definisi Fungsi dari Soal ---

print("--- Validasi Soal Integrasi ---")

# 1) Integral 1/x dari 1 ke 2, n=5
f1 = lambda x: 1/x
res1 = trapezoidal_rule(f1, 1, 2, 5)
print(f"1) Trapesium (n=5): {res1:.4f} (Doc: 0.6956)")

# 2 & 3) Integral sqrt(4+x^3) dari 0 ke 3, n=6
f2 = lambda x: np.sqrt(4 + x**3)
res2 = trapezoidal_rule(f2, 0, 3, 6)
res3 = simpsons_rule(f2, 0, 3, 6)
print(f"2) Trapesium (n=6): {res2:.4f} (Doc: 9.3301)")
print(f"3) Simpson (n=6): {res3:.4f} (Doc: 9.2787)")

# 4) Integral x^2 dari 0 ke 12, n=6
f4 = lambda x: x**2
res4 = midpoint_rule(f4, 0, 12, 6)
print(f"4) Midpoint (n=6): {res4:.0f} (Doc: 572)")

# 5) Integral sin^2(pi*x) dari 0 ke 1, n=3
f5 = lambda x: np.sin(np.pi * x)**2
res5 = midpoint_rule(f5, 0, 1, 3)
print(f"5) Midpoint (n=3): {res5:.4f} (Doc: 0.5000)")

# 18) Estimasi Pi (Integral 1/(1+x^2) * 4)
f18 = lambda x: 1/(1 + x**2)
res18 = trapezoidal_rule(f18, 0, 1, 4) * 4
print(f"18) Estimasi Pi (n=4): {res18:.4f} (Doc: 3.1312)")

# 35) Density Function
f35 = lambda x: (1/np.sqrt(2 * np.pi)) * np.exp(-x**2 / 2)
res35 = trapezoidal_rule(f35, 0, 0.4, 4)
print(f"35) Density Func (n=4): {res35:.4f} (Doc: 0.1554)")

# 45) Data Tabel (Custom Trapezoidal)
y_values = [125, 125, 120, 112, 90, 90, 95, 88, 75, 35, 0]
dx_45 = 100
area_45 = (dx_45 / 2) * (y_values[0] + y_values[-1] + 2 * sum(y_values[1:-1]))
print(f"45) Area Tanah: {area_45} m^2 (Doc: 89.250 atau 89250)")

--- Validasi Soal Integrasi ---
1) Trapesium (n=5): 0.6956 (Doc: 0.6956)
2) Trapesium (n=6): 9.3304 (Doc: 9.3301)
3) Simpson (n=6): 9.2792 (Doc: 9.2787)
4) Midpoint (n=6): 572 (Doc: 572)
5) Midpoint (n=3): 0.5000 (Doc: 0.5000)
18) Estimasi Pi (n=4): 3.1312 (Doc: 3.1312)
35) Density Func (n=4): 0.1553 (Doc: 0.1554)
45) Area Tanah: 89250.0 m^2 (Doc: 89.250 atau 89250)
